# DPA2上机实战

<a href="https://bohrium.dp.tech/notebook/62349a165b874acd951aeea947b2e17a  #DPA-SSE.ipynb" target="_blank"><img src="https://cdn.dp.tech/bohrium/web/static/images/open-in-bohrium.svg" alt="Open In Bohrium"/></a>

<div class="alert alert-info">

💭在阅读本notebook之前，建议读者先阅读DeePMD和DPA-1的相关教程，本文将不再详细介绍输入文件中的各项参数的含义：
1. [哥伦布训练营|DPA-1——固态电解质实战之模型训练&性质计算篇](https://nb.bohrium.dp.tech/detail/7145731165)
2. [快速开始 DeePMD-kit｜训练甲烷深度势能分子动力学模型](https://nb.bohrium.dp.tech/detail/1003)

</div>

## **1. 学习目标**
在本教程学习后，你将获得：
1. DPA-2基本原理和应用；
2. 以H2O-SCAN0数据集为例，进行DPA-2势函数模型训练实战：输入脚本解读；从头训 vs. 已有预训练模型微调；

## **2. DPA-2简介**
👂<b style='color:#021E66'> **迫不及待动手实践？可直接跳转至第3节~**

### **2.1 研究背景**
一直以来，势函数训练都在追求精度和效率的平衡。使用经典力场势函数方便快捷，但模拟精度难以更上一层楼；使用近来火热的AIMD(从头算分子动力学)，势函数精度获得大幅提升，但计算资源花费难以在大体系、长时间的场景落地。随着AI for science的发展，**机器学习手段使得训练高精度、高效率的势函数成为可能**(下图：分子动力学模拟对比)。在MLMD的新范式下，量子化学计算(QM)不再直接应用于AIMD，而是作为生成机器学习势函数(MLP)的数据集准备; 当然，AIMD的计算结果也可以作为初始数据集。

![image.png](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/13387/816de22eec6548c4aca8fe84cb81e4e8/image.png)

ref. [Machine learning-accelerated quantum mechanics-based atomistic simulations for industrial applications](https://link.springer.com/article/10.1007/s10822-020-00346-6)

然而，由于现有模型迁移能力不足、缺乏通用的大模型，面对一个新的复杂体系，要获得可用的、较为完备的势函数模型，科学家们基本上仍然需要获取大量计算数据并从头开始训练模型。随着电子结构数据的积累，类比计算机视觉（CV）或者自然语言处理（NLP）等其他人工智能领域的发展，**“预训练+少量数据微调”**是解决这个难题比较自然的想法。

为了实现这一范式，我们亟需一种具有**强迁移能力、能容纳元素周期表大多数元素**的模型结构。

### **2.2 研究方法**

DPA-2模型是继DPA-1之后对DP系列模型的又一次全面升级：

一方面是DPA-2使用了多任务训练策略（multi-task training），可以同时使用DFT设置不同的多个数据集进行预训练，在对下游任务进行微调（finetune）时，模型的主干（将构型空间和化学空间的表示进行编码的部分）将被保留，并在后边连接到一个或多个头部网络。因此，预训练的数据集和微调的数据集的标注方法不必完全相同。经过微调得到的模型参数量较多，直接应用于生产场景（如MD模拟）时可能会导致效率低下，为了解决这一问题，我们可以将模型蒸馏（distillation）为一个参数量较少的模型，既能保持下游任务的准确性，又能提高速度，从而可以进行大规模的模拟。（Figure1：预训练，微调，蒸馏的流程图）

![alt image.png](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/11461/10e2b52367624ea7a3a12cff12408aad/6RYF9tJpDX0msByrKa-EsA.png)

另一方面，DPA-2进一步改进了模型结构，对原子间的相互作用实现了更为充分的建模，通过在合金、半导体、电池材料和药物分子等数据集上的预训练，能够学习到更多隐藏的原子交互信息，极大提升了模型在包含不同构象、不同组分的数据集之间的迁移能力。我们将模型在18个不同的数据集上进行了预训练，并将此预训练模型在各种下游任务上进行了迁移学习，实验表明，与DPA-1模型相比，DPA-2预训练模型能进一步大幅降低下游任务训练所需数据量及训练成本、提高模型预测精度(Figure2:DPA-2模型结构示意图)。

![alt image.png](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/11461/10e2b52367624ea7a3a12cff12408aad/DhaNEFWOeNlTkk8MXgNSUg.png)

### **2.3 实验验证**

**数据集概览**

![alt image.png](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/11461/82b2a315aebe4108a1bf3d56bdc30e46/KAHsNGXyxCoQt5e59bccjA.png)

我们将各个数据集在预训练模型中的表示（descriptor）可视化（使用t-SNE方法），结果如下图所示：

![alt img_v3_025j_f2abcf80-d0cb-45b0-afd0-3b8b32ad2abg.jpg](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/11461/82b2a315aebe4108a1bf3d56bdc30e46/pXvCAh99X9Tw9-nL_CwRJg.jpeg)

**下游任务微调**

研究者利用下游数据集对 DPA-1、DPA-2 和 multitask pretrained DPA-2模型进行了采样效率测试，观察随着下游任务数据量的增加，收敛之后的能量和力的RMSE的变化趋势（Figure 3: 下游数据集的结果）。

从图中可以看出：

1.从头训练的DPA-2模型比DPA-1模型有更好的收敛精度，尤其是在数据量足够大的情况下，这一结果凸显了DPA-2模型结构的优越性。

2.当进行多任务预训练（multitask pretrain）时，经过微调的DPA-2模型可能会产生比从头训练的DPA-2模型低得多的曲线，尤其是在下游数据有限的情况下。在 H2O -SCAN0 等数据集上，即使是零点均方根误差（RMSE）也足够精确。

![alt image.png](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/11461/82b2a315aebe4108a1bf3d56bdc30e46/4Eufc2TfK0pqcakQ64AcjQ.png)

## **3. DPA2实战**
学习了理论知识后，让我们直接动手实践吧!
本节，我们将以数据集H20-SCAN0为例，开展DPA-2的从头训和微调训练。

<b style='color:#021E66'>注：本教程所使用的数据集源自[科学智能广场（AIS-Square)](https://aissquare.com/)，有更多模型和数据需求的同学赶快去探索一下吧~

### 3.1 内容介绍

In [15]:
cd /root/dpa2

/root/dpa2


In [17]:
!tree -L 2

.
├── data
│   └── H2O-scan0
├── finetune
│   └── input.json
├── from_scratch
│   └── input.json
└── pretrain_model
    └── model.pt

5 directories, 3 files


让我们一起看看教程文档包含的内容：

data: H2O-SCAN0数据集

finetune: 基于预训练模型的微调训练的目录，input.json为输入文件

from_scratch: DPA-2从头训练的目录，input.json为输入文件

pretrain_model: 预训练模型所在的目录，model.pt为预训练模型

### 3.2 输入脚本准备

从头训和模型微调训练的输入文件在此例子下是完全相同的：

In [18]:
cat from_scratch/input.json

{
 "_comment": "that's all",
 "model": {
  "type_embedding": {
   "neuron": [
    8
   ],
   "tebd_input_mode": "concat"
  },
  "type_map": [
   "H",
   "He",
   "Li",
   "Be",
   "B",
   "C",
   "N",
   "O",
   "F",
   "Ne",
   "Na",
   "Mg",
   "Al",
   "Si",
   "P",
   "S",
   "Cl",
   "Ar",
   "K",
   "Ca",
   "Sc",
   "Ti",
   "V",
   "Cr",
   "Mn",
   "Fe",
   "Co",
   "Ni",
   "Cu",
   "Zn",
   "Ga",
   "Ge",
   "As",
   "Se",
   "Br",
   "Kr",
   "Rb",
   "Sr",
   "Y",
   "Zr",
   "Nb",
   "Mo",
   "Tc",
   "Ru",
   "Rh",
   "Pd",
   "Ag",
   "Cd",
   "In",
   "Sn",
   "Sb",
   "Te",
   "I",
   "Xe",
   "Cs",
   "Ba",
   "La",
   "Ce",
   "Pr",
   "Nd",
   "Pm",
   "Sm",
   "Eu",
   "Gd",
   "Tb",
   "Dy",
   "Ho",
   "Er",
   "Tm",
   "Yb",
   "Lu",
   "Hf",
   "Ta",
   "W",
   "Re",
   "Os",
   "Ir",
   "Pt",
   "Au",
   "Hg",
   "Tl",
   "Pb",
   "Bi",
   "Po",
   "At",
   "Rn",
   "Fr",
   "Ra",
   "Ac",
   "Th",
   "Pa",
   "U",
   "Np",
   "Pu",
   "Am",
   "Cm",
   "Bk",

相比于DPA-1模型，DPA-2主要不同的参数集中在`descriptor`部分
```json
"descriptor": {
   "type": "hybrid",
   "hybrid_mode": "sequential",
   "list": [
    {
     "type": "se_atten",
     "sel": 120,
     "rcut_smth": 8.0,
     "rcut": 9.0,
     "neuron": [
      25,
      50,
      100
     ],
     "resnet_dt": false,
     "axis_neuron": 12,
     "seed": 1,
     "attn": 128,
     "attn_layer": 0,
     "attn_dotr": true,
     "attn_mask": false,
     "post_ln": true,
     "ffn": false,
     "ffn_embed_dim": 1024,
     "activation": "tanh",
     "scaling_factor": 1.0,
     "head_num": 1,
     "normalize": true,
     "temperature": 1.0,
     "add": "concat",
     "pre_add": true,
     "_comment": " that's all"
    },
    {
     "type": "se_uni",
     "sel": 40,
     "rcut_smth": 3.5,
     "rcut": 4.0,
     "nlayers": 12,
     "g1_dim": 128,
     "g2_dim": 32,
     "attn2_hidden": 32,
     "attn2_nhead": 4,
     "attn1_hidden": 128,
     "attn1_nhead": 4,
     "axis_dim": 4,
     "update_h2": false,
     "update_g1_has_conv": true,
     "update_g1_has_grrg": true,
     "update_g1_has_drrd": true,
     "update_g1_has_attn": true,
     "update_g2_has_g1g1": true,
     "update_g2_has_attn": true,
     "attn2_has_gate": true,
     "add_type_ebd_to_seq": false,
     "smooth": true,
     "_comment": " that's all"
    }
   ]
  },
```

相比于之前大家常用的se_e2_a描述子和DPA-1模型的se_atten描述子来说，DPA-2采用了se_attn和se_uni的混合描述子，其中两个描述子为串联关系，即se_attn描述子的输出作为se_uni描述子的输入。具体的模型构造可参考Figure 2。 

### 3.3 模型训练(从头训)

In [19]:
cd from_scratch

/root/dpa2/from_scratch


In [20]:
!dp_pt train input.json

2023-12-01 11:12:25.880822: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-01 11:12:25.880908: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-01 11:12:25.984780: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-01 11:12:26.194761: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-01 11:12:27.714354: W tensorflow/comp

### 3.4 模型微调训练

In [21]:
cd ../finetune

/root/dpa2/finetune


微调训练的命令增加了`--finetune ../pretrain_model/model.pt`选项，表示在模型`../pretrain_model/model.pt`的基础上微调。

除此之外，命令还增加了`-m H2O_H2O-PD`选项，原因是我们使用的数据集H2O-SCAN0与预训练数据集H2O_H2O-PD较为接近，所以我们使用预训练得到的H2O_H2O-PD的fitting net的参数来初始化我们微调的模型的fitting net的参数。

In [22]:
!dp_pt train input.json --finetune ../pretrain_model/model.pt -m H2O_H2O-PD

2023-12-01 11:14:28.921978: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-01 11:14:28.922049: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-01 11:14:28.923370: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-01 11:14:28.931242: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-01 11:14:29.940467: W tensorflow/comp

### 3.5 模型检验
现在，我们已经完成了DPA-2势函数的从头训和微调训练。让我们通过lcurve(学习率变化曲线输出文件）看下模型的表现吧！

📌注：由于节省时间，教程中的训练步仅为200步，没有到达一个收敛的结果；在真实训练场景中，记得调整至合适的训练时长~

In [23]:
cd /root/dpa2

/root/dpa2


想要直观对比从头训练和基于预训练模型的微调训练的lcurve差异？一键运行为大家准备好的可视化脚本试试~

In [24]:
import numpy as np
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(16,4))
data_scratch = np.genfromtxt("from_scratch/lcurve.out", names=True)
data_finetune = np.genfromtxt("finetune/lcurve.out", names=True)
for idx,ii in enumerate([1,3,5]):
    plt.subplot(1,3,idx+1)
    for name in data_scratch.dtype.names[ii:ii+1]:
        plt.plot(data_scratch["step"], data_scratch[name], label=f"scratch_{name}")
    for name in data_finetune.dtype.names[ii:ii+1]:
        plt.plot(data_finetune["step"], data_finetune[name], label=f"finetune_{name}")
    plt.legend()
    plt.xlabel("Step")
    plt.ylabel("Loss")
    #plt.xscale("symlog")
    plt.yscale("log")
    plt.grid()
plt.show()


<Figure size 1600x400 with 3 Axes>

根据lcurve的变化对比，我们可以看出基于预训练模型的微调训练具有更低的能量和力损失

## 4. 参考来源
DPA-2相关：
1. DPA-2 paper in preparation

DPA-1相关：
1. [DPA-1: Pretraining of Attention-based Deep Potential Model for Molecular Simulation](https://arxiv.org/abs/2208.08236)
2. [DeePMD-kit’s documentation](https://docs.deepmodeling.com/projects/deepmd/en/master/index.html)
3. [快速上手深度势能预训练模型 DPA-1](https://zhuanlan.zhihu.com/p/556201431)
4. [DPA-1: 共建覆盖元素周期表的预训练大模型](https://zhuanlan.zhihu.com/p/555832644#:~:text=DPA-1%E6%A8%A1%E5%9E%8B%E6%98%AF%E5%9F%BA%E4%BA%8EDP%E7%B3%BB%E5%88%97%E6%A8%A1%E5%9E%8B%E7%9A%84%E4%B8%80%E6%AC%A1%E5%85%A8%E9%9D%A2%E5%8D%87%E7%BA%A7%EF%BC%8C%E5%88%A9%E7%94%A8%E5%85%B3%E9%94%AE%E7%9A%84%20%E9%97%A8%E6%8E%A7%E6%B3%A8%E6%84%8F%E5%8A%9B%E6%9C%BA%E5%88%B6%EF%BC%88Gated%20Attention%20Machanism%EF%BC%89%EF%BC%8C%E5%AF%B9%E5%8E%9F%E5%AD%90%E9%97%B4%E7%9A%84%E7%9B%B8%E4%BA%92%E4%BD%9C%E7%94%A8%E5%AE%9E%E7%8E%B0%E4%BA%86%E6%9B%B4%E4%B8%BA%E5%85%85%E5%88%86%E7%9A%84%E5%BB%BA%E6%A8%A1,%EF%BC%8C%E9%80%9A%E8%BF%87%E5%9C%A8%E7%8E%B0%E6%9C%89%E6%95%B0%E6%8D%AE%E4%B8%8A%E7%9A%84%E8%AE%AD%E7%BB%83%EF%BC%8C%E8%83%BD%E5%A4%9F%E5%AD%A6%E4%B9%A0%E5%88%B0%E6%9B%B4%E5%A4%9A%E9%9A%90%E8%97%8F%E7%9A%84%E5%8E%9F%E5%AD%90%E4%BA%A4%E4%BA%92%E4%BF%A1%E6%81%AF%EF%BC%8C%20%E6%9E%81%E5%A4%A7%E6%8F%90%E5%8D%87%E4%BA%86%E6%A8%A1%E5%9E%8B%E5%9C%A8%E5%8C%85%E5%90%AB%E4%B8%8D%E5%90%8C%E6%9E%84%E8%B1%A1%E3%80%81%E4%B8%8D%E5%90%8C%E7%BB%84%E5%88%86%E7%9A%84%E6%95%B0%E6%8D%AE%E9%9B%86%E4%B9%8B%E9%97%B4%E7%9A%84%E8%BF%81%E7%A7%BB%E8%83%BD%E5%8A%9B%20%EF%BC%8C%E4%BB%8E%E8%80%8C%E4%B9%9F%E6%8F%90%E5%8D%87%E4%BA%86%E5%9C%A8%E6%95%B0%E6%8D%AE%E7%94%9F%E6%88%90%E6%97%B6%E7%9A%84%E9%87%87%E6%A0%B7%E6%95%88%E7%8E%87%EF%BC%9B%E5%B9%B6%E4%B8%94%E6%A8%A1%E5%9E%8B%E9%80%9A%E8%BF%87%20%E5%AF%B9%E5%85%83%E7%B4%A0%E4%BF%A1%E6%81%AF%E7%9A%84%E7%BC%96%E7%A0%81%EF%BC%8C%E6%8B%93%E5%B1%95%E4%BA%86%E5%AF%B9%E5%85%83%E7%B4%A0%E7%9A%84%E5%AE%B9%E9%87%8F%20%E3%80%82)
5. [张铎：DPA-1预训练模型介绍&上机实践](https://www.bilibili.com/video/BV1gD4y177DT/?spm_id_from=333.337.search-card.all.click&vd_source=ac172494d332ac871da890a8eb283526)

推荐阅读DPMD系列Notebook：
1. [哥伦布训练营|DPA-1——固态电解质实战之模型训练&性质计算篇](https://nb.bohrium.dp.tech/detail/7145731165)
2. [快速开始 DeePMD-kit｜训练甲烷深度势能分子动力学模型](https://nb.bohrium.dp.tech/detail/1003)
3. [从 DFT 到 MD｜超详细「深度势能」材料计算上手指南](https://nb.bohrium.dp.tech/detail/1007)
4. [使用深度势能分子动力学进行固态电解质研究实战](https://bohrium.dp.tech/notebook/d3dcf4144f6540f180f86847b5a4dfaa)

参考文献

1. Han Wang, Linfeng Zhang, Jiequn Han, and Weinan E. DeePMD-kit: A deep learning package for many-body potential energy representation and molecular dynamics. Comput. Phys. Comm., 228:178–184, 2018. doi:10.1016/j.cpc.2018.03.016.
2. Jinzhe Zeng, Duo Zhang, Denghui Lu, Pinghui Mo, Zeyu Li, Yixiao Chen, Marián Rynik, Li'ang Huang, Ziyao Li, Shaochen Shi, Yingze Wang, Haotian Ye, Ping Tuo, Jiabin Yang, Ye Ding, Yifan Li, Davide Tisi, Qiyu Zeng, Han Bao, Yu Xia, Jiameng Huang, Koki Muraoka, Yibo Wang, Junhan Chang, Fengbo Yuan, Sigbjørn Løland Bore, Chun Cai, Yinnian Lin, Bo Wang, Jiayan Xu, Jia-Xin Zhu, Chenxing Luo, Yuzhi Zhang, Rhys E. A. Goodall, Wenshuo Liang, Anurag Kumar Singh, Sikai Yao, Jingchao Zhang, Renata Wentzcovitch, Jiequn Han, Jie Liu, Weile Jia, Darrin M. York, Weinan E, Roberto Car, Linfeng Zhang, and Han Wang. DeePMD-kit v2: A software package for Deep Potential models. 2023. doi:10.48550/arXiv.2304.09409.
3. Huang J, Zhang L, Wang H, Zhao J, Cheng J, E W. Deep potential generation scheme and simulation protocol for the Li10GeP2S12-type superionic conductors. J Chem Phys. 2021;154(9):094703. doi:10.1063/5.0041849
4. https://docs.deepmodeling.com/projects/deepmd/en/master/index.html
5. https://github.com/deepmodeling/deepmd-kit